In [1]:
import numpy as np
import pandas as pd
import spacy

from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [2]:
nlp = spacy.load("en_core_web_md")
stop = stopwords.words('english')

In [3]:
a = np.load('./outfile.npy')
a.shape

(77, 2)

In [4]:
df = pd.DataFrame(a)
df.columns = ['movie','summary']
df.shape,df.head()

((77, 2),
              movie                                            summary
 0             1917  1917\nScene by Scene Breakdown\nBy Alexis Howe...
 1  A Monster Calls  A Monster Calls\nScene by Scene Breakdown\nby ...
 2    A Quiet Place  A Quiet Place\nScene by Scene Breakdown\nBy Ma...
 3      All is Lost  All Is Lost\nScene-by-Scene Breakdown\nWritten...
 4  American Hustle  American Hustle\nScene By Scene Breakdown\nBy ...)

In [5]:
def prep(x):
    def rem_up(i):
        if i.isupper():
            return i.title()
        else:
            return i
    x = x.replace('\n',' ')
    x = list(map(rem_up,x.split(' ')))
    x = ' '.join(x)
    return x

df['summary_preprocess'] = df['summary'].apply(lambda x: prep(x))

In [6]:
df['summary_doc'] = list(nlp.pipe(df['summary_preprocess'],n_process=20,batch_size=64))

In [7]:
df.head()

,movie,summary,summary_preprocess,summary_doc
0,1917,1917\nScene by Scene Breakdown\nBy Alexis Howe...,1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis..."
1,A Monster Calls,A Monster Calls\nScene by Scene Breakdown\nby ...,A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow..."
2,A Quiet Place,A Quiet Place\nScene by Scene Breakdown\nBy Ma...,A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,..."
3,All is Lost,All Is Lost\nScene-by-Scene Breakdown\nWritten...,All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd..."
4,American Hustle,American Hustle\nScene By Scene Breakdown\nBy ...,American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown..."


In [8]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [9]:
def extract_people2(doc):
    t_names = [ent.text for ent in doc.ents if ent.label_ =='PERSON']
    return t_names

In [10]:
def get_human_names(text):
    text = ' '.join([i for i in text.split()])
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)

    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        person.append(" ".join([leaf[0] for leaf in subtree.leaves() if leaf[1] =='NNP']))
    return person

In [11]:
def filter_names(t_names):
    t_names = [" ".join([i.orth_ for i in nlp(n) if i.pos_ in ['PROPN','NOUN']]) for n in t_names]
    t_names = list(filter(None,t_names))
    t_names = sorted(t_names,key=lambda x: (len(x.split(" ")),len(x)))
#     print(t_names,len(t_names))
    for i,v in enumerate(t_names):
        for s in t_names[i+1:]:
            if v in s and v!=s:
    #             print(s,s.split(" "))
                idx = t_names.index(s)
                t_names.remove(s)
                for k_idx, k in enumerate(s.split(" ")):
                    t_names.insert(idx+k_idx,k)
    #             t_names[idx] = s.split(" ")
    #             print(t_names,len(t_names))
#     print(t_names,len(t_names))
    t_names = [i for i in t_names if len(i)>2 and not any(c.isdigit() for c in i)]
    return Counter(t_names)

In [15]:
a_ = df['summary_doc'][0]
print(df['summary_doc'][0])
print('---------------------------------')

l2 = filter_names(extract_people2(a_))
l3 = filter_names(get_human_names(str(a_)))

print(l2,l3)
print('---------------------------------')

f_l = list(set(l2) & set(l3))
for i in f_l:
    print(f"{i}: (l2={l2[i]}), (l3={l3[i]})")
print('---------------------------------')
for i in list(set(l2)-set(f_l)):
    print(i,l2[i])
print('---------------------------------')
for i in list(set(l3)-set(f_l)):
    print(i,l3[i])

f_l = list(set(l2) & set(l3))
f_l_f = [i for i in f_l if max(l2[i],l3[i])>2]+\
        [i for i in list(set(l3)-set(f_l)) if l3[i]>3]+\
        [i for i in list(set(l2)-set(f_l)) if l2[i]>3]
for i in f_l_f:
    print(f"{i}: (l2={l2[i]}), (l3={l3[i]})")

1917 Scene by Scene Breakdown By Alexis Howell-Jones GoIntoTheStory.com 1–6 Ordinary World Opening Image: Distant thunder, as Blake and Schofield lie with their eyes shut, by a tree. Lance Corporal Blake (19) is ordered to pick a man and bring his kit for a mission. He chooses Schofield (early 20s) who’s next to him with his eyes shut — of similar rank. Schofield in script has a “Wound Stripe” — to indicate he was previously wounded in action — not obvious on screen, but info in script. As they walk, Blake reads his mail. Schofield didn’t get any but doesn’t seem to mind. They talk of the minimal food they have. Looking at the soldiers they’re passing in the trench they’ve entered, they surmise that something is up — but their Sergeant says it’s not the Big Push they think is coming. They’re told to go in to a Dugout and meet… 6–11 Inciting Incident General Erinmore (50s) who tells Blake that Blake’s brother and his regiment, the 2nd Devons — are walking into a trap (chasing the German

Counter({'Blake': 51, 'Schofield': 38, 'Mackenzie': 6, 'Smith': 5, 'Leslie': 3, 'Ecoust': 3, 'Lieutenant': 3, 'Ivins': 2, 'Devons': 2, 'Hepburn': 2, 'Erinmore': 2, 'Stevenson': 2, 'C.O.': 1, 'Truck': 1, 'German': 1, 'Muller': 1, 'Man Land': 1, 'Oak Tree': 1, 'Lock': 1, 'Midpoint': 1, 'Cherry Blossoms': 1, 'Richards': 1, 'Alexis Howell Jones': 1, 'Lance': 1, 'Corporal': 1}) Counter({'Schofield': 77, 'Blake': 41, 'Mackenzie': 10, 'Smith': 5, 'Ecoust': 5, 'Line': 3, 'Leslie': 3, 'Captain': 3, 'Lauri': 2, 'Baumer': 2, 'Muller': 2, 'Hepburn': 2, 'Trenches': 2, 'Stevenson': 2, 'Ivins': 2, 'Raf': 1, 'Alexis': 1, 'Lock': 1, 'Trench': 1, 'D Companies': 1, 'Wound Stripe': 1, 'Scene Breakdown': 1, 'Doubts': 1, 'Cherry Blossoms': 1, 'Orchard': 1, 'Colonel': 1, 'Lance': 1, 'Corporal': 1})
---------------------------------
Mackenzie: (l2=6), (l3=10)
Leslie: (l2=3), (l3=3)
Lance: (l2=1), (l3=1)
Muller: (l2=1), (l3=2)
Hepburn: (l2=2), (l3=2)
Ecoust: (l2=3), (l3=5)
Ivins: (l2=2), (l3=2)
Schofield: (l2=

In [51]:
def extract_characters(x,verbose=False):
    l2 = filter_names(extract_people2(x))
    l3 = filter_names(get_human_names(str(x)))
    f_l = list(set(l2) & set(l3))
    if verbose:
        print(l2)
        print(l3)
        print(f_l)
    f_l_f = [i for i in f_l if max(l2[i],l3[i])>2]+\
            [i for i in list(set(l3)-set(f_l)) if l3[i]>3]+\
            [i for i in list(set(l2)-set(f_l)) if l2[i]>3]
    return f_l_f

In [20]:
df['characters'] = df['summary_doc'].apply(lambda x: extract_characters(x))

In [ ]:
df.to_csv('./movie_w_characters.csv',index=None)

# load back movie characters

In [3]:
df = pd.read_csv('./movie_w_characters.csv')

In [34]:
df.head()

,movie,summary,summary_preprocess,summary_doc,characters,summary_resolved
0,1917,1917\nScene by Scene Breakdown\nBy Alexis Howe...,1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis...","[Leslie, Smith, Mackenzie, Ecoust, Blake, Scho...",1917 Scene by Scene Breakdown By Alexis Howell...
1,A Monster Calls,A Monster Calls\nScene by Scene Breakdown\nby ...,A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow...","[Mum, Harry, Conor, Grandma, Lily, Dad]",A Monster Calls Scene by Scene Breakdown by An...
2,A Quiet Place,A Quiet Place\nScene by Scene Breakdown\nBy Ma...,A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,...","[Beau, Lee, Marcus, Abbott, Regan, Evelyn, Black]",A Quiet Place Scene by Scene Breakdown By Mark...
3,All is Lost,All Is Lost\nScene-by-Scene Breakdown\nWritten...,All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd...",[],All Is Lost Scene-by-Scene Breakdown Written b...
4,American Hustle,American Hustle\nScene By Scene Breakdown\nBy ...,American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown...","[Dolly, Richie, Stoddard, Greensley, Amado, Pe...",American Hustle Scene By Scene Breakdown By Jo...


In [10]:
df['characters']=df['characters'].apply(eval)

# Neuralcoref

In [5]:
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

/Users/vibhavagarwal/miniconda2/envs/basic_ml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/Users/vibhavagarwal/miniconda2/envs/basic_ml/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)


In [13]:
nlp.pipe_names

['tagger', 'parser', 'ner', 'neuralcoref']

In [14]:
df['summary_doc'] = df['summary_preprocess'].apply(lambda x:nlp(x))

In [6]:
doc = nlp(df['summary_preprocess'][10])

In [28]:
df['summary_resolved'] = df['summary_doc'].apply(lambda x: x._.coref_resolved)

In [38]:
df.head()

,movie,summary,summary_preprocess,summary_doc,characters,summary_resolved
0,1917,1917\nScene by Scene Breakdown\nBy Alexis Howe...,1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis...","[Leslie, Smith, Mackenzie, Ecoust, Blake, Scho...",1917 Scene by Scene Breakdown By Alexis Howell...
1,A Monster Calls,A Monster Calls\nScene by Scene Breakdown\nby ...,A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow...","[Mum, Harry, Conor, Grandma, Lily, Dad]",A Monster Calls Scene by Scene Breakdown by An...
2,A Quiet Place,A Quiet Place\nScene by Scene Breakdown\nBy Ma...,A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,...","[Beau, Lee, Marcus, Abbott, Regan, Evelyn, Black]",A Quiet Place Scene by Scene Breakdown By Mark...
3,All is Lost,All Is Lost\nScene-by-Scene Breakdown\nWritten...,All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd...",[],All Is Lost Scene-by-Scene Breakdown Written b...
4,American Hustle,American Hustle\nScene By Scene Breakdown\nBy ...,American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown...","[Dolly, Richie, Stoddard, Greensley, Amado, Pe...",American Hustle Scene By Scene Breakdown By Jo...


In [47]:
nlp.remove_pipe('neuralcoref')
nlp.has_pipe('neuralcoref')

False

In [48]:
df['summary_r_doc'] = list(nlp.pipe(df['summary_resolved'],n_process=20,batch_size=64))

In [49]:
df.head()

,movie,summary,summary_preprocess,summary_doc,characters,summary_resolved,summary_r_doc
0,1917,1917\nScene by Scene Breakdown\nBy Alexis Howe...,1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis...","[Leslie, Smith, Mackenzie, Ecoust, Blake, Scho...",1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis..."
1,A Monster Calls,A Monster Calls\nScene by Scene Breakdown\nby ...,A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow...","[Mum, Harry, Conor, Grandma, Lily, Dad]",A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow..."
2,A Quiet Place,A Quiet Place\nScene by Scene Breakdown\nBy Ma...,A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,...","[Beau, Lee, Marcus, Abbott, Regan, Evelyn, Black]",A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,..."
3,All is Lost,All Is Lost\nScene-by-Scene Breakdown\nWritten...,All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd...",[],All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd..."
4,American Hustle,American Hustle\nScene By Scene Breakdown\nBy ...,American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown...","[Dolly, Richie, Stoddard, Greensley, Amado, Pe...",American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown..."


In [52]:
df['characters_nc'] = df['summary_r_doc'].apply(lambda x: extract_characters(x))

In [53]:
df.head()

,movie,summary,summary_preprocess,summary_doc,characters,summary_resolved,summary_r_doc,characters_nc
0,1917,1917\nScene by Scene Breakdown\nBy Alexis Howe...,1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis...","[Leslie, Smith, Mackenzie, Ecoust, Blake, Scho...",1917 Scene by Scene Breakdown By Alexis Howell...,"(1917, Scene, by, Scene, Breakdown, By, Alexis...","[Blake, Smith, Mackenzie, Lance, Ecoust, Schof..."
1,A Monster Calls,A Monster Calls\nScene by Scene Breakdown\nby ...,A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow...","[Mum, Harry, Conor, Grandma, Lily, Dad]",A Monster Calls Scene by Scene Breakdown by An...,"(A, Monster, Calls, Scene, by, Scene, Breakdow...","[Lily, Dad, Mum, Grandma, Harry, Conor, Mr. Cl..."
2,A Quiet Place,A Quiet Place\nScene by Scene Breakdown\nBy Ma...,A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,...","[Beau, Lee, Marcus, Abbott, Regan, Evelyn, Black]",A Quiet Place Scene by Scene Breakdown By Mark...,"(A, Quiet, Place, Scene, by, Scene, Breakdown,...","[Beau, Marcus, Lee, Evelyn, Regan, Abbott, Black]"
3,All is Lost,All Is Lost\nScene-by-Scene Breakdown\nWritten...,All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd...",[],All Is Lost Scene-by-Scene Breakdown Written b...,"(All, Is, Lost, Scene, -, by, -, Scene, Breakd...",[]
4,American Hustle,American Hustle\nScene By Scene Breakdown\nBy ...,American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown...","[Dolly, Richie, Stoddard, Greensley, Amado, Pe...",American Hustle Scene By Scene Breakdown By Jo...,"(American, Hustle, Scene, By, Scene, Breakdown...","[Rosalyn, Elway, Musane, Amado, Sydney, Irv, P..."


In [60]:
set(df['characters_nc'][10]) - (set(df['characters'][10]) & set(df['characters_nc'][10]))

{'Jack Russell', 'Julius Rosenberg', 'escorts'}

In [62]:
df.to_csv('./movie_w_characters_nc.csv',index=None)

In [63]:
df['summary_r_doc'][10]

Beginners Scene-by-Scene Breakdown By Ali Coad GoIntoTheStory.com Page 1: Dead flowers on a window sill. Page 1: Oliver and Oliver old Jack Russell named Arthur clean and empty Oliver and his old Jack Russell father’s home. Oliver flushes all kids of medicine down the toilet. Oliver looks at an old picture of Oliver father, Hal, in a jean jacket. Pages 1–2: Flashback. Celebration of Hal with fireworks. A man named Andy records it. Page 2: Back to present. Oliver and his old Jack Russell leave the house. Page 2: Drive down Sunset Boulevard. Pages 2–3: Oliver shows Arthur around Oliver home, introducing Oliver to various rooms. Pages 3–4: Flashback. Narrated Sequence. Still black and white and color photographs flash by: the sun, the stars, Yosemite, George W. Bush, people kissing, faces laughing, faces crying. Photos of Oliver’s parents getting married, having Oliver. Oliver Vo of descriptions of the stills. We learn Oliver mom died of cancer. Pages 4–5: Flashback continues. Oliver Vo c